# Multiprocessing

**multiprocessing** es un módulo de Python que nos permite manejar varios procesos al mismo tiempo, a diferencia de threading que nos permitia manipular procesos livianos que no tenian espacio ni direcciones de memoria dedicados, sino que todos compartian un solo hilo (similar a JavaScript), multiprocessing nos permite crear nuevos procesos y poder llevar atareas mas robustas y realmente manejar multitareas, esto con ayuda del sistema operativo ya que no es python el que se encarga de adminitrar los procesos sino el sistema operativo como tal.

---

# la clase Process 

**Process** es una clase similar a Thread the threading que nos permite crear nuevos procesos y sus atributos y métodos son similares a los que ya vimos de Thread, **Process** tiene la siguiente estructura

`class multiprocessing.Process(group=None, target=None, name=None, args=(), kwargs={}, daemon=None)`

y cada uno de sus parámetros tiene lña siguiente función

- ``group`` = no hace nada solo es usado para seguir la misma estructura de Thread
- ``target`` = es el objeto invocable que se enviara puede ser un método o función
- ``name`` = es el nombre que le asignas, es muy util en tareas de depuración
- ``args`` = los argumentos enviados al obj invocable ( viene en formato de tupla )
- ``kwargs`` = un diccionario con los argumentos que serán enviados al obj invocable
- ``daemon`` = define si el proceso será un daemon process, un daemon process no podrá crear nuevos hijos

# Métodos y atributos de la clase Process

- `start()` = es el método que inicia el proceso
- `join([timeout])` = permite enlazar el priceso a otro que debera temrinarse para ejecutarse, si se envia el parametro timeout (float) ese sera el tiempo que esperará a que el otro proceso termine o de lo contrario ya no lo ejecutará
- `is_alive()` = nos permite saber si el proceso sigue activo o no
- `terminate()` = termina el proceso en cuestion, es decir que envia la señal de terminado para que el se encargue de realizar toda la tarea
- `kill()` = hace lo mismo que terminado pero forza al proceso a terminar si no lo hace lo elimina de una
- `close()` = Cierra el proceso y libera todos los recursos asociados, una vez que un procesos es cerrado la mayoria de sus métodos y atributos enviara `ValueError` 
- `name` = nombre con el cual identifican al proceso
- `daemon` = si el proceso es daemon o no
- `pid` = es el Process ID del proceso en cuestion
- `exitcode` = es el codigo con el que termino la tarea, envia None en caso de que no haya terminado aun


# Uso de Process

In [4]:
from multiprocessing import Process
from time import sleep

def worker(msg):
    for i in range(0, 10):
        print(msg, end='', flush=True)
        sleep(1)

print('Starting')

t2 = Process(target=worker, args='A')
t3 = Process(target=worker, args='B')
t4 = Process(target=worker, args='C')

t2.start()
t3.start()
t4.start()

print('Done')


Starting
Done


# Alternative Ways to Start a Process

existen tres maneras de iniciar un proceso, y estos se definen con el método ``set_start_method()`` cuyo valor se pasa en formato string y sus valores son:

- ``spawn`` = es el valor por defecto de windows y lo que hace es crear un nuevo interprete de Python en donde ejecutara el método ``run()`` con los datos enviados desde un principio, se puede usar tanto para windows, linux y mac OS
- `fork` = este es usado para clonar un nuevo interprete del proceso que esta creando un hijo, de tal forma que aunque son procesos separados podrán tener la misma información disponible en el inicio, esta es el valor por defecto de Linux y esposible usarlo en Mac OS también
- `forkserver` = funciona similar a forks pero cada vez que crea un proceso hijo, llama al proceso padre apra pedirle un clon del interprete, esta dispinible para Linux

un ejemplo de uso sería

In [2]:
from multiprocessing import Process
from multiprocessing import set_start_method
from time import sleep
import os


def worker(msg):
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())
    for i in range(0, 10):
        print(msg, end='', flush=True)
        sleep(1)


def main():
    print('Starting')
    print('Root application process id:', os.getpid())
    set_start_method('spawn')
    t = Process(target=worker, args='A')
    t.start()
    print('Done')


if __name__ == '__main__':
    main()


Starting
Root application process id: 22584
Done


# Using a Pool

La creación de procedimientos tiene un costo alto de recursos, por eso **multiprocessing** nos ofrece una clase llamada **Pool** que nos permite reutilizar procesos antes de terminarlos, así nos ahorramos ese gasto de tiempo y recursos.

la clase Pool tiene la siguiente sintaxis

```python
class multiprocessing.pool.Pool(processes, initializer, initargs, maxtasksperchild, context)
```

- ``processes`` = es el numero de workers que se usarán, en caso de None, se usará el valor de de ``os.cpu_coun()``
- ``initializer`` = un objeto invocable
- ``initargs`` = los argumentos que son enviados al obj invocable
- ``maxtasksperchild`` = define el numero de tareas que ejecutará un worker antes de ser reemplazado por uno nuevo, en caso de None cada worker durará lo que dure el Pool
- ``context`` = permite especificar el contexto usado para iniciar los process workers

Esta clase nos permite definir procesos que estarán a dispoción del objeto, el cual se encargara de realizar la administración y asignación de tareas a estos, podemos verlo mas claro en el siguiente grafico

![grafico](./images/img9.png)

# modo de uso de los métodos de Pool

La mas emocionante de Pool es que es muy sencillo utilizarlo con ayuda de sus métodos, uno de ellos es ``map()``

`pool.map(func, iterable, chunksize=None)`

- `func` = es un objeto invocable
- `iterable` = son los datos o valores que se le deben de enviar al obj invocable
- `chunksize` = hace referencia a la cantidad de datos que se le deben de enviar de ``iterable`` a ``func`` en caso que sea menor al iterable se ejecutara x veces la función hasta completar los iterables

podemos verlo en el siguiente código

In [ ]:
from multiprocessing import Pool

def worker(x):
    print('In worker with: ', x)
    return x * x


def main():
    with Pool(processes=4) as pool:
        print(pool.map(worker, [0, 1, 2, 3, 4, 5]))
    
if __name__ == '__main__':
    main()

> Ejecute el programa ``pool-base.py`` para verlo en funcionamiento

# Exchanging Data Between Processes

Para intercambiar datos entre procesos, el módulo multiprocessing nos ofrece la función ``Pipe()``, esta función se encarga de habilitar una interfaz entre dos procesos, por defecto la comunicación es de dos caminos, es decir que ambos procesos pueden comunicarse entre si, aunque esto puede ser modificado, la comunicación se hace através de los metodos ``send()`` y ``recv()``, podemos visualizar el funcionamiento en el siguiente grafico

![grafico](./images/img10.png)


In [ ]:
from multiprocessing import Process, Pipe
from time import sleep

def worker(conn):
    print('Worker - started now sleeping for 1 second')
    sleep(1)
    print('Worker - sending data via Pipe')
    conn.send('Hello!')
    print('Worker - closing worker end of connection')
    conn.close()
    
def main():
    print('Main - starting, creating the pipe')
    main_connection, worker_connection = Pipe()
    print('Main - setting up the process')
    p = Process(target=worker, args=[worker_connection])
    print('Main - Starting the process')
    p.start()
    print('Main - Wait for a response from the child process')
    print(main_connection.recv())
    print('Main - Closing parent process end of conenction')
    main_connection.close()
    print('Main - Done')
    
if __name__ == '__main__':
    main()

> Ejecutar el programa ``pool-pipe.py`` para verlo en funcionamiento

